<font size = "3">
QTM 151 Section 4
Group Member: Nate Hu, Max Jiang, Garry Zhu

Introduction (Background Analysis):

Formula 1 is a premier motorsport competition recognized globally for its high-speed racing, cutting-edge technology, and iconic drivers and teams. This project aims to delve into the world of Formula 1, exploring various aspects of the competition, such as circuit characteristics, race results, driver performance, constructor success, and race statuses. Understanding the intricacies of this sport is relevant not only to fans and enthusiasts but also to stakeholders such as teams, sponsors, and broadcasters who can leverage insights to make informed decisions and improve various aspects of the competition.

Undertsanding the Schema:
In this project, we analyze a comprehensive Formula 1 dataset, focusing on different levels of the competition, including circuit, race, driver, constructor, and status. The goal is to extract meaningful insights and patterns that can help us answer key questions about the sport, such as identifying the most successful drivers or constructors, understanding the impact of circuit characteristics on race outcomes, and assessing the factors that contribute to the overall performance of a driver or a team. The project is structured into different sections, each addressing a specific level of the competition. The results of our analysis will be presented in a clear and concise manner, supported by visualizations and data-driven conclusions that cater to both Formula 1 aficionados and novices alike.

Question:

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
circuits = pd.read_csv("data_raw/circuits.csv")
constructors_results = pd.read_csv("data_raw/constructor_results.csv")
constructors_standings = pd.read_csv("data_raw/constructor_standings.csv")
constructors = pd.read_csv("data_raw/constructors.csv")
driver_standing = pd.read_csv()